In [1]:
import base64
import requests
import json
from config import sportskey
from config import password
from config import gkey
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import linregress
import os
from ohmysportsfeedspy import MySportsFeeds
import gmaps

msf = MySportsFeeds(version="2.0")
msf.authenticate(sportskey, "MYSPORTSFEEDS")

In [2]:
output = msf.msf_get_data(league='nhl',season='2018-2019-regular',feed='seasonal_player_stats',format='json')


In [3]:
# records amount of records/players
num_records = len(output["playerStatsTotals"])

In [4]:
offense_df = pd.DataFrame(columns=['Name', 'Age', 'Birth City', 'Birth Country', 'Birth Lat', 'Birth Lng', 'College', 'College Lat', 'College Lng', 'Position', 'Points', 'Ability Score'], index = np.arange(num_records))
offense_df

,Name,Age,Birth City,Birth Country,Birth Lat,Birth Lng,College,College Lat,College Lng,Position,Points,Ability Score
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# loops through every row of empty dataframe and adds the position of each player
for x in range(num_records):
    offense_df.loc[x, 'Position'] = output["playerStatsTotals"][x]["player"]["primaryPosition"]

# removes any player that is a goalie
offense_df = offense_df.loc[offense_df["Position"] != "G"]
offense_df = offense_df.reset_index(drop=True)
offense_df


,Name,Age,Birth City,Birth Country,Birth Lat,Birth Lng,College,College Lat,College Lng,Position,Points,Ability Score
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LW,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LW,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LW,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RW,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,NaN,NaN
1401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RW,NaN,NaN
1402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LW,NaN,NaN
1403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,NaN,NaN


In [6]:
num_offense_records = len(offense_df)

In [7]:
for x in range(num_offense_records):
    try:
        offense_df.loc[x, 'Name'] = output["playerStatsTotals"][x]["player"]["firstName"] + " " + output["playerStatsTotals"][x]["player"]["lastName"]
        offense_df.loc[x, 'Age'] = output["playerStatsTotals"][x]["player"]["age"]
        offense_df.loc[x, 'Birth City'] = output["playerStatsTotals"][x]["player"]["birthCity"]
        offense_df.loc[x, 'Birth Country'] = output["playerStatsTotals"][x]["player"]["birthCountry"]
        offense_df.loc[x, 'College'] = output["playerStatsTotals"][x]["player"]["college"]
        offense_df.loc[x, 'Points'] = output["playerStatsTotals"][x]["stats"]["scoring"]["points"]
    except KeyError:
        print("can't find stat for player" + str(x))

offense_df

,Name,Age,Birth City,Birth Country,Birth Lat,Birth Lng,College,College Lat,College Lng,Position,Points,Ability Score
0,Justin Abdelkader,33,"Muskegon, MI",USA,NaN,NaN,None,NaN,NaN,LW,19,NaN
1,Pontus Aberg,26,Stockholm,Sweden,NaN,NaN,None,NaN,NaN,LW,19,NaN
2,Pontus Aberg,26,Stockholm,Sweden,NaN,NaN,None,NaN,NaN,LW,6,NaN
3,Vitaly Abramov,21,Chelyabinsk,Russia,NaN,NaN,None,NaN,NaN,RW,0,NaN
4,Noel Acciari,28,"Johnston, RI",USA,NaN,NaN,None,NaN,NaN,C,14,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1400,Mathieu Olivier,23,"Levis, QC",USA,NaN,NaN,None,NaN,NaN,D,0,NaN
1401,Gustav Olofsson,25,Boras,Sweden,NaN,NaN,None,NaN,NaN,RW,0,NaN
1402,Ryan Olsen,26,"Delta, BC",Canada,NaN,NaN,None,NaN,NaN,LW,0,NaN
1403,Cal O'Reilly,33,"Toronto, ON",Canada,NaN,NaN,None,NaN,NaN,D,0,NaN
